In [ ]:
!pip install sagemaker -q -U

In [ ]:
!pip install datasets -q -U

In [2]:
import sagemaker

sess = sagemaker.Session()
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::905847418383:role/service-role/AmazonSageMaker-ExecutionRole-20211005T160629
sagemaker bucket: sagemaker-us-east-1-905847418383
sagemaker session region: us-east-1


In [7]:
!wget https://raw.githubusercontent.com/TQuad/turkish-nlp-qa-dataset/master/dev-v0.1.json

--2021-10-06 14:12:18--  https://raw.githubusercontent.com/TQuad/turkish-nlp-qa-dataset/master/dev-v0.1.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 458494 (448K) [text/plain]
Saving to: ‘dev-v0.1.json.1’

dev-v0.1.json.1     100%[===================>] 447.75K  --.-KB/s    in 0.005s  

2021-10-06 14:12:18 (92.9 MB/s) - ‘dev-v0.1.json.1’ saved [458494/458494]



In [8]:
!wget https://raw.githubusercontent.com/TQuad/turkish-nlp-qa-dataset/master/train-v0.1.json

--2021-10-06 14:12:29--  https://raw.githubusercontent.com/TQuad/turkish-nlp-qa-dataset/master/train-v0.1.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3078146 (2.9M) [text/plain]
Saving to: ‘train-v0.1.json.1’

train-v0.1.json.1   100%[===================>]   2.94M  --.-KB/s    in 0.01s   

2021-10-06 14:12:29 (251 MB/s) - ‘train-v0.1.json.1’ saved [3078146/3078146]



In [3]:
bucket = sess.default_bucket()

In [4]:
!aws s3 cp data/train.json s3://$bucket/qa-tr/data/train.json
!aws s3 cp data/val.json s3://$bucket/qa-tr/data/val.json

upload: data/train.json to s3://sagemaker-us-east-1-905847418383/qa-tr/data/train.json
upload: data/val.json to s3://sagemaker-us-east-1-905847418383/qa-tr/data/val.json


In [41]:
from sagemaker.huggingface import HuggingFace

# hyperparameters, which are passed into the training job
hyperparameters={
    'model_name_or_path': 'dbmdz/bert-base-turkish-uncased',
#     'dataset_name':'squad',
    'train_file': '/opt/ml/input/data/train/train.json',
    'validation_file': '/opt/ml/input/data/val/val.json',
    'do_train': True,
    'do_eval': False,
    'fp16': True,
    'per_device_train_batch_size': 4,
    'per_device_eval_batch_size': 4,
    'num_train_epochs': 2,
    'max_seq_length': 384,
#     'max_steps': 100,
    'pad_to_max_length': True,
    'doc_stride': 128,
    'output_dir': '/opt/ml/model'
}

# configuration for running training on smdistributed Data Parallel
# distribution = {'smdistributed':{'dataparallel':{ 'enabled': True }}}

# git configuration to download our fine-tuning script
# git_config = {'repo': 'https://github.com/marshmellow77/transformers.git','branch': 'v4.10.0'}

# instance configurations
instance_type = 'ml.p3.16xlarge'
instance_count = 1
volume_size = 200

In [42]:
huggingface_estimator = HuggingFace(entry_point='run_qa.py',
#                                     source_dir='./examples/pytorch/question-answering',
                                    source_dir='./scripts',
#                                     git_config=git_config,
#                                     metric_definitions=metric_definitions,
                                    instance_type=instance_type,
                                    instance_count=instance_count,
                                    volume_size=volume_size,
                                    role=role,
                                    transformers_version='4.10',
                                    pytorch_version='1.9',
                                    py_version='py38',
#                                     distribution= distribution,
                                    hyperparameters= hyperparameters,
                                    disable_profiler=True,
                                   )

In [43]:
huggingface_estimator.fit({'train': 's3://sagemaker-us-east-1-905847418383/qa-tr/data/', 'val': 's3://sagemaker-us-east-1-905847418383/qa-tr/data/'})

2021-10-12 05:07:28 Starting - Starting the training job...
2021-10-12 05:07:32 Starting - Launching requested ML instances............
2021-10-12 05:09:30 Starting - Preparing the instances for training.........
2021-10-12 05:11:21 Downloading - Downloading input data
2021-10-12 05:11:21 Training - Downloading the training image...........................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-10-12 05:15:48,985 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-10-12 05:15:49,060 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-10-12 05:15:49,067 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-10-12 05:15:49,729 sagemaker-training-toolkit INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "val": "/opt/ml/input/d

In [24]:
from sagemaker.huggingface import HuggingFaceModel
import sagemaker 

role = sagemaker.get_execution_role()

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data="s3://sagemaker-us-east-1-905847418383/huggingface-pytorch-training-2021-10-11-15-01-25-153/output/model.tar.gz",  # path to your trained sagemaker model
   role=role, # iam role with permissions to create an Endpoint
   transformers_version="4.10", # transformers version used
   pytorch_version="1.9", # pytorch version used
   py_version="py38", # python version of the DLC
)

In [44]:
# deploy model to SageMaker Inference
predictor = huggingface_estimator.deploy(
   initial_instance_count=1,
   instance_type="ml.m5.xlarge"
)

---------------!

In [45]:
data = {
"inputs": {
    "question": "Ne zaman avare bir hayata başladı?",
    "context": """ABASIYANIK, Sait Faik. Hikayeci (Adapazarı 23 Kasım 1906-İstanbul 11 Mayıs 1954). \
İlk öğrenimine Adapazarı’nda Rehber-i Terakki Mektebi’nde başladı. İki yıl kadar Adapazarı İdadisi’nde okudu.\
İstanbul Erkek Lisesi’nde devam ettiği orta öğrenimini Bursa Lisesi’nde tamamladı (1928). İstanbul Edebiyat \
Fakültesi’ne iki yıl devam ettikten sonra babasının isteği üzerine iktisat öğrenimi için İsviçre’ye gitti. \
Kısa süre sonra iktisat öğrenimini bırakarak Lozan’dan Grenoble’a geçti. Üç yıl başıboş bir edebiyat öğrenimi \
gördükten sonra babası tarafından geri çağrıldı (1933). Bir müddet Halıcıoğlu Ermeni Yetim Mektebi'nde Türkçe \
gurup dersleri öğretmenliği yaptı. Ticarete atıldıysa da tutunamadı. Bir ay Haber gazetesinde adliye muhabirliği\
yaptı (1942). Babasının ölümü üzerine aileden kalan emlakin geliri ile avare bir hayata başladı. Evlenemedi.\
Yazları Burgaz adasındaki köşklerinde, kışları Şişli’deki apartmanlarında annesi ile beraber geçen bu fazla \
içkili bohem hayatı ömrünün sonuna kadar sürdü."""
    }
}
predictor.predict(data)

{'score': 0.24896939098834991,
 'start': 752,
 'end': 775,
 'answer': 'Babasının ölümü üzerine'}